Rabin Karp algoritması bir pattern eşleştirme algoritmasıdır. Biz bu örnekte bir yazının içinde sözcük araması yapacağız. Ancak iki boyutlu bir sinyalde belli bir örüntü aramak gibi daha kompleks işlemlerde de kullanmak mümkündür. Bunu yapmak için , verilen küme (bu örnkete yazı) ve verilen örüntü (bu örnekte aradığımız küçük sözcük) için hash değeri hesaplayacağız. Hash değeri verilen parçaları matematiksel olarak ifade etmemizi sağlar. hash değerleri kıyaslamak tek tek harf kıyaslamaktan daha verimlidir. 

text = AAAAAAAAAH  9 A ve 1 H  
pattern = AAAAAAAAH  8A ve bir H  

eğer direkt olarak ilk sırada harf karşılaştırmaya başlarsak 8 tane a eşleştikten sonra A ve H kıyaslayıp eşit olmadığını görecek bir sonraki sıraya geçip kıayslama yapacak ve 9 karşılaştırmadan sonra eşleşmeyi bulacak.

Eğer hash hesabı yaparsak her şey daha rahat olur. 

Peki hash hesabı nedir?


Her karakteri belirten sayısal bir değer vardır 

In [ ]:
print(ord('A')) 

In [ ]:
print(ord('a'))

In [ ]:
print(ord('1')) 

stringi oluşturan karakterlerin değerini toplamak basit bir hesap çözümü

In [1]:
def simple_hash(string):

    hash_value = 0
    for char in string:
        hash_value += ord(char)  #it returns an integer representing the Unicode code of the character passed to it.
    return hash_value

In [2]:
input_string = "hello"
hash_value = simple_hash(input_string)
print("Hash value of '{}': {}".format(input_string, hash_value))

Hash value of 'hello': 532


peki ama bu bize niye yarasın ki? Rolling hash diye bişi yapcaz

ascii de 256 muhtemel değer var bunlar büyük küçük harfler ve semboller için 0dan 255e değişen değerlerdir .

bizde arama yaptığımız metin (text) içinde aradığımız örüntümüz(pattern) ile aynı boyutta parçalar alıp hashlarına bakacağız. Eğer hashlar aynı değerdeyse oturup tek tek karakterleri karşılaştıuracağız.


hashları karşılaştırmak tek tek karakter karşılaştırmaktan çok daha kolaydır. Bu yüzdenbn gereksiz harf karşılaştırmaktan kaçınmak çok önemlidir

Gereksiz harf karşılaştırma miktarını en aza indirmek için en iyi yöntem hash değerlerinin olabildiğince farklı olmasından emin olmaktır.

örneğin 

abca texti içinde 

abc ve bca patternleri,  eğer sadece karkter değerlerini toplarsak aynı hashı üretecek.bu iki kere kontrol demek.

bunun yerine  eğer pozisyondan da etkilenen bir hash üretimi yaparsak farklı değerler alınmasını sağlarız.

Bu mantık aslında rolling hash ve check sum olarakta bilinir. Hatta check sum emthodu interntten yaptığınız büyük indirmelerin ( mesela bir linux distrosu) doğruluğunu kontrol eder. Eğer hatalı bir indirme yaptıysan (indirme sırasında bozuk bir dosya inmesi paket kaybı olabilir düşük bir ihtimal olsada cihazını riske atacak bir indirmede, işletim sistemi güncellemesi gibi, kontrol etmek iyidir.)

basamağa bağlı hash değeri üreten bir sistem yapalım

aşağıda arama yaptığımız patternin basamak sayısına bağlı farklı değerler oluşturan bir sistem yapacağız

In [8]:
pattern = "abcadacac"
q = 17 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
for i in range(len(pattern)-1):
		h = (h*d) % q
		print(h)

12
8
11
13
3
2
7
16


Yukarda gördüğün sistem 1 basamak için 12 2 basmak için 8 üç basamak için 11 değerini üretti ve bizim final değerimiz 16 oldu mevcut uzunluk için 

asal sayı olarak 17 seçmiştik peki ya 13 seçseydik?

In [10]:
pattern = "abcadacac"
q = 13 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
for i in range(len(pattern)-1):
		h = (h*d) % q
		print(h)

9
3
1
9
3
1
9
3


gördüğün gibi hash kendini tekrar etti ancak 17 için özel sayıalar oluştu. asal sayıyı duruma göre seçmek biraz deneyim ister ancak büyük asal sayılar daha başarılı olur
farklı basamak uzunlukları için aynı hash değerini veriyor. bu bizim 1 ve 4 basamaklı pattern farkının hasha bakarak anlamamızı engeller çünkü ikiside 9 değerinde

In [ ]:
# Following program is the python implementation of
# Rabin Karp Algorithm given in CLRS book

# d is the number of characters in the input alphabet
d = 256

# pat -> pattern
# txt -> text
# q -> A prime number


def search(pat, txt, q):
	M = len(pat)
	N = len(txt)
	i = 0
	j = 0
	p = 0 # hash value for pattern
	t = 0 # hash value for txt
	h = 1

	# The value of h would be "pow(d, M-1)%q"
	for i in range(M-1):
		h = (h*d) % q

	# Calculate the hash value of pattern and first window
	# of text
	for i in range(M):
		p = (d*p + ord(pat[i])) % q
		t = (d*t + ord(txt[i])) % q

	# Slide the pattern over text one by one
	for i in range(N-M+1):
		# Check the hash values of current window of text and
		# pattern if the hash values match then only check
		# for characters one by one
		if p == t:
			# Check for characters one by one
			for j in range(M):
				if txt[i+j] != pat[j]:
					break
				else:
					j += 1

			# if p == t and pat[0...M-1] = txt[i, i+1, ...i+M-1]
			if j == M:
				print("Pattern found at index " + str(i))

		# Calculate hash value for next window of text: Remove
		# leading digit, add trailing digit
		if i < N-M:
			t = (d*(t-ord(txt[i])*h) + ord(txt[i+M])) % q

			# We might get negative values of t, converting it to
			# positive
			if t < 0:
				t = t+q


# Driver Code
if __name__ == '__main__':
	txt = "GEEKS FOR GEEKS"
	pat = "GEEK"

	# A prime number
	q = 101

	# Function Call
	search(pat, txt, q)
